## Import modules

In [1]:
# !pip install Dask

In [2]:
import numpy as np
import open3d as o3d
from sklearn.ensemble import RandomForestClassifier
import pdal
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plyfile import PlyData, PlyElement
import gc
from itertools import chain
from sklearn.metrics import confusion_matrix
import os
import joblib # Use to save model
from datetime import datetime, timezone
# from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from glob import glob
# import dask
from sklearn.model_selection import GridSearchCV
sns.set()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# ROOT_PLY = """/home/sspiegel/CapstoneData/Paris/Toronto_3D/L001.ply"""

In [4]:
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
from dask.utils import format_bytes


In [5]:
ROOT = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/training/total_training.parquet"""

In [6]:
ff = ['EigenSum_radius1',
 'omnivariance_radius1',
 'entropy_radius1',
 'linearity_radius1',
 'planarity_radius1',
 'sphericity_radius1',
 'curvature_radius1',
 'verticality1_radius1',
 'verticality2_radius1',
 'HeightVariance_radius1',
 'HeightRange_radius1',
 'count_radius1',
 'EigenSum_radius2',
 'omnivariance_radius2',
 'entropy_radius2',
 'linearity_radius2',
 'planarity_radius2',
 'sphericity_radius2',
 'curvature_radius2',
 'verticality1_radius2',
 'verticality2_radius2',
 'HeightVariance_radius2',
 'HeightRange_radius2',
 'count_radius2',
 'EigenSum_radius3',
 'omnivariance_radius3',
 'entropy_radius3',
 'linearity_radius3',
 'planarity_radius3',
 'sphericity_radius3',
 'curvature_radius3',
 'verticality1_radius3',
 'verticality2_radius3',
 'HeightVariance_radius3',
 'HeightRange_radius3',
 'count_radius3',
 'EigenSum_radius4',
 'omnivariance_radius4',
 'entropy_radius4',
 'linearity_radius4',
 'planarity_radius4',
 'sphericity_radius4',
 'curvature_radius4',
 'verticality1_radius4',
 'verticality2_radius4',
 'HeightVariance_radius4',
 'HeightRange_radius4',
 'count_radius4',
 'EigenSum_radius5',
 'omnivariance_radius5',
 'entropy_radius5',
 'linearity_radius5',
 'planarity_radius5',
 'sphericity_radius5',
 'curvature_radius5',
 'verticality1_radius5',
 'verticality2_radius5',
 'HeightVariance_radius5',
 'HeightRange_radius5',
 'count_radius5',
 'EigenSum_radius6',
 'omnivariance_radius6',
 'entropy_radius6',
 'linearity_radius6',
 'planarity_radius6',
 'sphericity_radius6',
 'curvature_radius6',
 'verticality1_radius6',
 'verticality2_radius6',
 'HeightVariance_radius6',
 'HeightRange_radius6',
 'count_radius6']

In [7]:
cls = ['label']

In [11]:
df = dd.read_parquet(ROOT, engine='fastparquet')

NotImplementedError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: Fastparquet engine is not supported

In [9]:
df = dd.from_pandas(df,npartitions = 1000)

In [10]:
gc.collect()

80

In [19]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [10]:
gridParams = {
    "n_estimators" : [50],
    "max_depth" : [40]
}

In [11]:
from dask_ml.model_selection import GridSearchCV as DaskGrid

In [20]:
grd = DaskGrid(RandomForestClassifier(),param_grid=gridParams,n_jobs=-1,scoring = 'f1')

In [21]:
X = df[ff]
y = df[cls]

In [22]:
# client.close()

In [23]:
client = Client()

# cluster = LocalCluster()
# client = Client(cluster)

2025-11-30 15:34:38,031 - distributed.scheduler - ERROR - Task ('repartitiontofewer-5e628094b608da02321065c719a034b0', 0) has 16.13 GiB worth of input dependencies, but worker tcp://127.0.0.1:34147 has memory_limit set to 11.79 GiB.
2025-11-30 15:34:50,348 - distributed.scheduler - ERROR - Task ('repartitiontofewer-5e628094b608da02321065c719a034b0', 0) has 16.13 GiB worth of input dependencies, but worker tcp://127.0.0.1:34147 has memory_limit set to 11.79 GiB.


In [24]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 94.31 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40275,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:33085,Total threads: 4
Dashboard: http://127.0.0.1:45515/status,Memory: 11.79 GiB
Nanny: tcp://127.0.0.1:41501,


In [27]:
rf = RandomForestClassifier(n_jobs=-1)

In [ ]:
with joblib.parallel_backend('dask'):
    rf.fit(X,y)

In [26]:
client.close()

In [18]:
grd

,estimator,RandomForestClassifier()
,param_grid,"{'max_depth': [40], 'n_estimators': [50]}"
,scoring,'f1_weighted'
,iid,True
,refit,True
,cv,None
,error_score,'raise'
,return_train_score,False
,scheduler,None
,n_jobs,-1
,cache_cv,True


In [4]:
# from dask.distributed import Client, LocalCluster

## Load in features

## Output model

In [32]:
OUTS = f"""{datetime.now().strftime("%Y_%m_%d")}_Trained_cylinder_RF_Lille.joblib"""

joblib.dump(rf, f"""/home/sspiegel/CapstoneData/Paris/RF_models/{OUTS}""")

['/home/sspiegel/CapstoneData/Paris/RF_models/2025_11_15_Trained_cylinder_RF_Lille.joblib']